In [1]:
import hail as hl
import varspark.hail as vshl
vshl.init()

using variant-spark jar at '/home/jupyter-chrysanialim/.conda/envs/py37/lib/python3.7/site-packages/varspark/jars/variant-spark_2.11-0.5.0-a0-dev5-all.jar'
23/02/09 06:20:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/09 06:20:57 WARN Hail: This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.3.
  Compatibility is not guaranteed.
Running on Apache Spark version 3.1.3
SparkUI available at http://dto-bgsi-galaxy-dep.asia-southeast2-a.c.dto-bgsi-dev.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.74-0c3a74d12093
LOGGING: writing to /home/jupyter-chrysanialim/hail-20230209-0620-0.2.74-0c3a74d12093.log


In [2]:
from hail.plot import show
from pprint import pprint
# hl.plot.output_notebook()

In [3]:
# vcf file was converted from plink
data = hl.import_vcf('alzheimer_vcf_2.vcf', contig_recoding={'23': 'X', '24' : 'Y', '25': 'MT'})

In [4]:
data.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        PR: bool
    }
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [5]:
# label & data format covariate file
labels = hl.import_table('covariates_hail.csv', delimiter=';',
                        types={'FID':'str', 'IID':'str', 'Diagnosis':'float64', 'Age':'float64', 'APOE':'float64'},
                        key='IID')

2023-02-09 06:22:14 Hail: INFO: Reading table without type imputation
  Loading field 'FID' as type str (user-supplied)
  Loading field 'IID' as type str (user-supplied)
  Loading field 'Diagnosis' as type float64 (user-supplied)
  Loading field 'Age' as type float64 (user-supplied)
  Loading field 'APOE' as type float64 (user-supplied)


In [6]:
labels.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'FID': str 
    'IID': str 
    'Diagnosis': float64 
    'Age': float64 
    'APOE': float64 
----------------------------------------
Key: ['IID']
----------------------------------------


In [7]:
mt = data.annotate_cols(labels = labels[data.s])
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'labels': struct {
        FID: str, 
        Diagnosis: float64, 
        Age: float64, 
        APOE: float64
    }
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        PR: bool
    }
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [8]:
mt.count()
# removed 778 SNPs in chr 3,4,19,MT due to 'not within the range of GRCh37'
# all of chrMT SNPs are removed

2023-02-09 06:22:28 Hail: INFO: Coerced sorted dataset              (3 + 1) / 4]


(355836, 364)

## Logistic regression - Hail

In [10]:
# logistic regression: Diagnosis
gwas_logreg_diagnosis = hl.logistic_regression_rows(test='score',
                                y=mt.labels.Diagnosis,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0, mt.labels.Age, mt.labels.APOE],
                                 pass_through=[mt.rsid])

2023-02-09 06:27:53 Hail: INFO: Coerced sorted dataset              (3 + 1) / 4]


FatalError: HailException: No complete samples: each sample is missing its phenotype or some covariate

Java stack trace:
is.hail.utils.HailException: No complete samples: each sample is missing its phenotype or some covariate
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:11)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:11)
	at is.hail.utils.package$.fatal(package.scala:77)
	at is.hail.stats.RegressionUtils$.getPhenosCovCompleteSamples(RegressionUtils.scala:103)
	at is.hail.methods.LogisticRegression.execute(LogisticRegression.scala:37)
	at is.hail.expr.ir.functions.WrappedMatrixToTableFunction.execute(RelationalFunctions.scala:50)
	at is.hail.expr.ir.TableToTableApply.execute(TableIR.scala:2826)
	at is.hail.expr.ir.TableMapRows.execute(TableIR.scala:1903)
	at is.hail.expr.ir.Interpret$.apply(Interpret.scala:26)
	at is.hail.backend.spark.SparkBackend.$anonfun$pyPersistTable$2(SparkBackend.scala:495)
	at is.hail.expr.ir.ExecuteContext$.$anonfun$scoped$3(Execute

In [ ]:
manhattan_logreg_diagnosis = hl.plot.manhattan(gwas_logreg_diagnosis.p_value, hover_fields=dict(rs=gwas_logreg_diagnosis.rsid))
show(manhattan_logreg_diagnosis)

In [ ]:
hl.plot.output_notebook() # WARNING! running this will make notebook unsavable

p = hl.plot.manhattan(gwas_logreg_diagnosis.p_value, hover_fields=dict(rs=gwas_logreg_diagnosis.rsid))
show(p)

## Random Forest - VariantSpark

In [ ]:
rf_model = vshl.random_forest_model(y=mt.hipster.label,
                    x=mt.GT.n_alt_alleles(), 
                    covariates={'age':mt.hipster.age, 'PC0':mt.hipster.PC0, 'PC1':mt.hipster.PC1, 'PC2':mt.hipster.PC2})
rf_model.fit_trees(500, 100)

In [ ]:
# Capture variant importances
print(rf_model.oob_error())
impTable = rf_model.variable_importance()
impTable.show(3)

In [ ]:
# Show the covariates importances
covImpTable = rf_model.covariate_importance()
covImpTable.show(4)

In [ ]:
# Join hail and VariantSpark results (this is only needed here to get the RSID's)
gwas_with_imp = gwas.join(impTable)

In [ ]:
import varspark.hail.plot as vshlplt

p = vshlplt.manhattan_imp(gwas_with_imp.importance, 
                            hover_fields=dict(ri=gwas_with_imp.rsid),
                            significance_line = None)
show(p)

In [ ]:
# Compare logistc regression values vs. rf importance
p = hl.plot.scatter(x=-hl.log10(gwas_with_imp.p_value),
                    y=gwas_with_imp.importance, 
                    xlabel = '-log10(p-value)',
                    ylabel = 'gini importance',
                    hover_fields=dict(rs=gwas_with_imp.rsid, loc=gwas_with_imp.locus))
show(p)

## Linear regression - Hail

In [ ]:
gwas = hl.linear_regression_rows(y=mt.label.score,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])

In [ ]:
gwas.show()

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)